In [1]:
import pandas as pd
import json
import subprocess

## Preparation

In [2]:
# 1. filter GREIN_data for human GSE IDs
grein_df = pd.read_csv("../demo/GREIN_data.csv")
human_df = grein_df[grein_df['Species'] == "Homo sapiens"]
human_ids = human_df['GEO accession']
human_ids.to_csv("../demo/metadata_human_ID.tsv", sep="\t", index=False, header=False)

# 2. Extract titles from metadata_human
with open("../demo/metadata_human.json") as f:
    metadata = json.load(f)

human_titles = [metadata[gse]["Title"] for gse in human_ids if gse in metadata]
pd.DataFrame(human_titles).to_csv("../demo/metadata_human_title.tsv", sep="\t", index=False, header=False)

## Prediction

In [3]:
# Preprocess
subprocess.run([
    "python", "../src/preprocess.py",
    "-input", "../demo/metadata_human_title.tsv",
    "-out", "../demo/metadata_human_title_processed.tsv"
])

# Generate embeddings
subprocess.run([
    "python", "../src/embedding_lookup_table.py",
    "-input", "../demo/metadata_human_title_processed.tsv",
    "-out", "../demo/metadata_human_title_embedding.npz"
])

# Predict for one human model (e.g. MONDO_0000315)
subprocess.run([
    "python", "../src/predict.py",
    "-input", "../demo/metadata_human_title_processed.tsv",
    "-id", "../demo/metadata_human_ID.tsv",
    "-input_embed", "../demo/metadata_human_title_embedding.npz",
    "-train_embed", "../data/disease_desc_embedding.npz",
    "-model", "../bins/MONDO_0000315__model.pkl",
    "-out", "../results/"
])

preprocessing text: 100%|████████████████| 3395/3395 [00:00<00:00, 8026.22it/s]


get unique words
there are 7032 unique words in the given descriptions
generate embedding by cpu


generate embeddings using cpu: 100%|███████| 7032/7032 [02:41<00:00, 43.62it/s]


saving output...


/home/rfu_smith_edu/.local/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


loading data
took 1.20 s to load data
predicting labels
took 4.37 s to predict
retrieving predictive words
took 0.10 s to retrieve predictive words
saving output
took 0.09 min to load, predict, retrieve predictive words and save MONDO_0000315 for 3395 instances


CompletedProcess(args=['python', '../src/predict.py', '-input', '../demo/metadata_human_title_processed.tsv', '-id', '../demo/metadata_human_ID.tsv', '-input_embed', '../demo/metadata_human_title_embedding.npz', '-train_embed', '../data/disease_desc_embedding.npz', '-model', '../bins/MONDO_0000315__model.pkl', '-out', '../results/'], returncode=0)